In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import os
import numpy as np

import matplotlib.pyplot as plt
from src.models.ig_cnn import ig_cnn_model, get_cnn_dataset
from src.tftools.callbacks.t2_callbacks import InformationGainCallback, ImagesCallback

* [How connect spectrum](www.fourmilab.ch/documents/specrend/)
* [Python library](https://python-colormath.readthedocs.io/en/latest/color_objects.html#spectralcolor)

In [3]:
hparams = {
    'image_path' : '../data/raw/still_life/sample_22.mat',
    'model_def'  : 'C-25-3-sigmoid-valid,C-25-1-sigmoid-same',
    'input_size' : 3 ,
    'log_name' : 'C3'
}

In [4]:
# Parameters
logdir = os.path.join('..', 'logs', hparams['log_name'])


# Input Data
inputs, outputs, pad_output, nopad_input = get_cnn_dataset(hparams['image_path'],
                                                           input_size=hparams['input_size'])
# Model Definition
model = ig_cnn_model(hparams['model_def'])
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

# Data split
training_set_size = 2500
perm = np.random.permutation(inputs.shape[0])
ins = inputs[perm[:training_set_size], :]
outs = outputs[perm[:training_set_size], :]

# TODO: Check log dir existence

callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir),
             InformationGainCallback(input_image=nopad_input,
                                     target_image=pad_output,
                                     logdir=logdir),
             ImagesCallback(input_image=nopad_input,
                            target_image=pad_output,
                            logdir=logdir),
             hp.KerasCallback(logdir,hparams),
             # NOTE: This callback has to be last
             tf.keras.callbacks.EarlyStopping(patience=6, min_delta=10 ** -5),
 ]

history = model.fit(ins,
                    outs,
                    epochs=100,
                    validation_split=0.2,
                    verbose=1,
                    callbacks=callbacks
                    )

NameError: name 'wavelength2rgb' is not defined